<a href="https://colab.research.google.com/github/naoya1110/jetbot_road_following_by_classification/blob/main/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Following by Classification - Data Collection



## Introduction

In the collision avoidance example, we made the JetBot moving around without collisions by using a CNN model that can distinguish `free` and `broked` situations from camera inputs.

In this week, we try to make the JetBot to follow a road by modifying the method we used for the collision avoidance. We are going to make a CNN model that can classify `go forward`, `turn left`, `turn right` situations, instead of `free` and `brocked` situations.

## Preparation

### Display Camera Input
Create a camera instance and an image widget, and then make a link between them.

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### Create Robot Class Instance
Create an instance of `Robot` class named `robot` so that we can contol the JetBot.

In [ ]:
from jetbot import Robot
robot = Robot()

### Create Data Directories
Create directories for storing the image data.

In [ ]:
import os

forward_dir = 'dataset/forward'
left_dir = 'dataset/left'
right_dir = 'dataset/right'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(forward_dir)
    os.makedirs(left_dir)
    os.makedirs(right_dir)
except FileExistsError:
    print('Directories not created because they already exist')

Directories not created because they already exist


### Create Controller

Create a GUI controller with forward, left and right buttons.

In [ ]:
try:
    del forward_button
    del left_button
    del right_button

except:
    pass

# base button layout
button_layout = widgets.Layout(width='74px', height='50px', align_self='center')

# create counter widgets
forward_count = widgets.IntText(layout=button_layout, value=len(os.listdir(forward_dir)))
left_count = widgets.IntText(layout=button_layout, value=len(os.listdir(left_dir)))
right_count = widgets.IntText(layout=button_layout, value=len(os.listdir(right_dir)))

# create acution button widgets
forward_button = widgets.Button(description='forward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# layout counters and action button widgets
counters = widgets.HBox([left_count, forward_count, right_count])
actions = widgets.HBox([left_button, forward_button, right_button])
counters_box = widgets.VBox([actions, counters], align_self='center')

# display controller box
display(counters_box)

### Define Action Functions
Define functions to make robot move forward, turn left and turn right.

In [ ]:
import time

def move_forward():
    robot.set_motors(0.3, 0.3)
    time.sleep(0.2)
    robot.stop()

def move_left():
    robot.set_motors(0, 0.3)
    time.sleep(0.2)
    robot.stop()

def move_right():
    robot.set_motors(0.3, 0)
    time.sleep(0.2)
    robot.stop()

### Define Data Save Functions
Define functions to save snapshots when you make the JetBot to move forward, turn left and turn right.

In [ ]:
from uuid import uuid1
from datetime import datetime


def save_snapshot(directory):
    now = datetime.now()
    now = now.strftime("%Y%m%d_%H%M%S")
    image_path = os.path.join(directory, now + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_forward():
    global forward_dir, forward_count
    save_snapshot(forward_dir)
    move_forward()
    forward_count.value = len(os.listdir(forward_dir))

def save_left():
    global left_dir, left_count
    save_snapshot(left_dir)
    move_left()
    left_count.value = len(os.listdir(left_dir))

def save_right():
    global right_dir, right_count
    save_snapshot(right_dir)
    move_right()
    right_count.value = len(os.listdir(right_dir))

    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
forward_button.on_click(lambda x: save_forward())
left_button.on_click(lambda x: save_left())
right_button.on_click(lambda x: save_right())

## Data Collection

Now we are ready to perform data collection.

In [ ]:
display(widgets.VBox([image, counters_box]))

## Stop Camera
After you finished the data collection, stop camera.

In [ ]:
camera.stop()

## Zip Dataset
Make `dataset.zip` so that we can upload it to Google Colab for training a model.

In [ ]:
!zip -r -q dataset.zip dataset


---

## *Delete All Dataset (Danger!)*

Next commands delete all of your collected data in ``dataset`` directory.

In [ ]:
import shutil
shutil.rmtree("dataset")